# Case 13: SQL I

## *How can we optimize our sales of financial products?*

## Goals (2 min)

By the end of ths case, you will be familiar with databases. Specifically, you will learn the differences among the major types of databases and the different database management systems available. Basic SQL queries will also be introduced.

You will also be exposed to the technical jargon of databases. While you probably will not use these terms on a daily basis, they will give you a more holistic understanding of the data engineering discipline and facilitate conversations between yourself and other data engineers.

## Introduction (5 min)

**Business Context.** You are a data analyst at a large financial services firm that sells a diverse portfolio of products. In order to make these sales, the firm relies on a call center where sales agents make calls to current as well as prospective customers. The company would like you to dive into their data to devise strategies to increase their revenue or reduce their costs. Specifically, they would like to double down on their most reliable customers, and to cut out sales agents whom are not producing outcomes.

**Business Problem.** The business would like to answer the following questions: **"What types of customers are most likely to buy our product? And which of my sales agents are the most/least productive?**

**Analytical Context.** The data is split across 3 tables: "Agents", "Calls", and "Customers", which sit on CSV files. Unlike previous cases though, we will first be reading these CSV files into a SQLite database created within Python. You will learn how this database differs from CSV files and how to interact with it using SQL to extract useful insights.

The case is sequenced as follows: you will (1) learn the fundamentals of databases and SQL; (2) use SQL `SELECT` statements to identify potentially interesting customers; and (4) use SQL aggregation functions to compute summary statistics on your agents and identify the most/least productive ones.

## What is this "SQL" thing? (8 min)

In this case, we will be writing SQL queries using the [`SQLAlchemy`](https://www.sqlalchemy.org/) package in Python. This allows you to directly interface with relational databases without exiting the Python environment, while using syntax that is identical to what you would write outside of Python. Run the code below to set up this framework:

In [3]:
import pandas as pd
from sqlalchemy import create_engine, text

#maximum number of rows to display
pd.options.display.max_rows = 10

engine=create_engine('sqlite://')
df = pd.read_csv('customer.csv').to_sql('customer', engine, if_exists='replace', index=False)
df = pd.read_csv('agent.csv').to_sql('agent', engine, if_exists='replace', index=False)
df = pd.read_csv('call.csv').to_sql('call', engine, if_exists='replace', index=False)

def runQuery(sql):
    result = engine.connect().execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

In [4]:
query_robin = """SELECT COUNT(*) FROM customer"""
runQuery(query_robin)

,COUNT(*)
0,1000


The columns in each of the tables are as follows:

**agent.csv**:
- **AgentID**: the primary key of the table (more on this below)
- **Name**: the name of the agent

**call.csv**:
- **CallID**: the primary key of the table
- **AgentID**: a foreign key (more on this below) to the agents table of the agent who made the call
- **CustomerID**: a foreign key to the customers table of the customer who is being called
- **PickedUp**: a Boolean that is 1 if the customer picked up and 0 if they did not
- **Duration**: integer of the duration of the call
- **ProductSold**: a Boolean that is 1 if the agent made a sale and 0 if they did not

**customer.csv**:
- **CustomerID**: the primary key of the table
- **Name**: the name of the customer
- **Occupation**: the occupation of the customer. 'Unemployed' if no occupation
- **Email**: the email of the customer
- **Company**: the company that the customer works for
- **PhoneNumber**: the phone number of the customer
- **Age**: the age of the customer


The above database structure can be visualized as below. This is called an **Entity Relationship (ER) Diagram**, denoting the tables present in the database, the columns in the tables, and the relations among the tables:

![ER Diagram](images/database_schema.png)

The above diagram gives a good overview of how the schema is structured and how the data is interconnected.

## Finding potentially interesting customer cohorts (20 min)

The most important thing you will ever do in SQL is extract a subset of the data from a SQL table based on a set of rules. This is accomplished using the **`SELECT`** statement and the following syntax:

![Select Anatomy](./images/select_anatomy.png)

To translate the above diagram into words:

1. Start with the keyword `SELECT`
2. Follow with the names of the columns you want to select, separated by commas (alternatively, you can use the `*` symbol to indicate you wish to select all columns)
3. Follow with the keyword `FROM`
4. Finish with the name of the table you wish to select data from
5. Optionally, you can use the `WHERE` clause to only return results which satisfy certain conditions (similar to how code within Python `if...then` blocks only execute if the associated conditions are true)

Since the firm wants to dig deeper into its customers, let's start by pulling some of their data out of our files; namely, information about customers who are not unemployed (and therefore are more likely to buy from us).

### Exercise 1: (5 min)

Write a query that selects the customer ID and name from the `Customer` table, only showing results for customers who are not unemployed. Remember to write your query as a multi-line string (enclosed within a pair of triple quotes `"""`) and pass it to the `runQuery()` function defined in the framework above to check your work!

**Answer**. One possible solution is given below:

```SQL
SELECT CustomerID, Name
FROM Customer
WHERE Occupation != 'Unemployed'
```

In [6]:
query1 = """SElECT CustomerID, Name
FROM Customer
WHERE Occupation != 'Unemployed'"""
runQuery(query1)

,CustomerID,Name
0,1,Michael Gonzalez
1,2,Amanda Wilson
2,3,Robert Thomas
3,4,Eddie Hall
4,6,Maria Johnson
...,...,...
755,994,Ruben Steele
756,995,Ashley Young
757,996,Mr. Steven Smith
758,997,Mark Smith


Of course, for names, it's sensible to try to list them in alphabetical order. SQL allows us to do this rather easily with the `ORDER BY` statement. This is then followed by a comma-separated list of columns on which you want to order your results (columns that come first take priority in the subsequent ordering). Optionally, you can then append the keyword `ASC` or `DESC` (short for ascending and descending, respectively) after each column to determine the ordering type (e.g. alphabetical or reverse-alphabetical for a string column).

We can also use the `AS` statment to change the name of a column returned by your query. However, this change is only temporary and is only valid for that particular query. For example, we can rename the `Name` column to `CustomerName` and order it alphabetically. This operation is known as **aliasing**:

```SQL
SELECT CustomerID, Name AS CustomerName
FROM Customer
WHERE Occupation != 'Unemployed'
ORDER BY CustomerName
```

In [9]:
query2 = """SELECT CustomerID, Name AS CustomerName
FROM Customer
WHERE Occupation != 'Unemployed'
ORDER BY CustomerName"""
runQuery(query2)

,CustomerID,CustomerName
0,900,Aaron Gutierrez
1,622,Aaron Rose
2,226,Adam Ward
3,786,Alan Chambers
4,985,Alan Mitchell
...,...,...
755,699,Willie Greene
756,715,Yesenia Wright
757,952,Yolanda White
758,421,Zachary Ruiz


This is a great first step; however, while producing the list of customers that are not unemployed, you inevitably spend a lot of time looking at the different professions your customers have and realize how often engineers appear in your database. You know that engineering jobs tend to command higher salaries these days, so you decide to try to extract a list of all the unique types of engineering jobs that are represented in your database. To ensure that you don't get duplicate job titles in your query results, you'll need to write the keyword `DISTINCT` immediately after `SELECT` in your query.

### Exercise 2: (5 min)

Write a query which produces a list, in alphabetical order, of all the distinct occupations in the `Customer` table that contain the word "Engineer".

**Hint:** The `LIKE` operator can be used when you want to look for similar values. It is included as part of a `WHERE` clause. It needs to be complemented with the `%` symbol, which is a wild card that represents zero, one, or multiple characters. For example, one valid `WHERE` clause utilizing the `LIKE` operator is `WHERE Name LIKE 'Matt%'`, which would return any results where the person's name starts with the word "Matt"; e.g. "Matt" or "Matteo" or "Matthew", etc.

**Answer.** One possible solution is given below:

```SQL
SELECT DISTINCT Occupation
FROM Customer
WHERE Occupation LIKE '%Engineer%'
ORDER BY Occupation
```

In [16]:
query3 = """SELECT DISTINCT Occupation
FROM Customer
WHERE Occupation LIKE '%Engineer%'
ORDER BY Occupation"""
runQuery(query3)

,Occupation
0,Chemical engineer
1,Electrical engineer
2,"Engineer, aeronautical"
3,"Engineer, agricultural"
4,"Engineer, automotive"
...,...
24,"Engineer, production"
25,"Engineer, site"
26,"Engineer, structural"
27,"Engineer, technical sales"


Now, one of your marketing colleagues tells you that people who are 30 or older will have a higher probability of buying your product (presumably because by that point they have more disposable income and savings). You don't want to take your colleague's word for granted, so you decide not to completely ignore people under 30, and instead add that information to the report regarding the person’s age, so that the agent making the subsequent call can decide how they want to use that information. However, due to privacy concerns, you also cannot share the person's exact age.

### Exercise 3: (5 min)

Write a query that retuns the customer ID, their name, and a column `Over30` containing "Yes" if the customer is more than 30 years of age and "No" if not.

**Hint:** You will need to use the `CASE-END` clause. The `CASE-END` clause can be used to evaluate conditional statements and returns a value once a condition is met (similar to an if-then-else clause in Python). If no conditions are true, it returns the value in the ELSE clause (or NULL if there is no ELSE statement). For example:

```SQL
CASE
    WHEN Name = "Matt" THEN 'Yes'
    WHEN Name = "Matteo" THEN 'Maybe'
    ELSE 'No'
END
```

**Answer.** One possible solution is given below:

```SQL
SELECT 
    CustomerID, 
    Name,
    CASE
        WHEN Age >= 30 THEN 'Yes'
        WHEN Age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM Customer
ORDER BY Name DESC
```

In [17]:
query4 = """SELECT CustomerID, Name,
    CASE
        WHEN Age >= 30 THEN 'Yes'
        WHEN Age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM Customer
ORDER BY Name DESC"""
runQuery(query4)

,CustomerID,Name,Over30
0,392,Zachary Wilson,Yes
1,986,Zachary Stevenson,No
2,421,Zachary Ruiz,Yes
3,18,Zachary Howe,No
4,883,Zachary Anderson,No
...,...,...,...
995,65,Adam Jimenez,No
996,622,Aaron Rose,No
997,145,Aaron Mcintyre,No
998,461,Aaron Hendrix,No


Let's now modify Exercise 3 so that the query only returns customers who work in an engineering profession:

```SQL
SELECT CustomerID, Name,
    CASE
        WHEN Age >= 30 THEN 'Yes'
        WHEN Age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM Customer
WHERE Occupation LIKE '%Engineer%'
ORDER BY Name DESC
```

In [18]:
query5 = """SELECT CustomerID, Name,
    CASE
        WHEN Age >= 30 THEN 'Yes'
        WHEN Age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM Customer
WHERE Occupation LIKE '%Engineer%'
ORDER BY Name DESC"""
runQuery(query5)

,CustomerID,Name,Over30
0,421,Zachary Ruiz,Yes
1,952,Yolanda White,No
2,699,Willie Greene,Yes
3,973,William Jackson,Yes
4,966,William Garcia,No
...,...,...,...
356,918,Alison Vaughan,Yes
357,568,Alice Lee,No
358,432,Alexis Riddle,No
359,985,Alan Mitchell,Yes


## Investigating customer conversion rates (20 min)

In order to validate whether our hypotheses about engineers and age are true (for example, engineers exhibit higher product sales conversion rates, and perhaps engineers over 30 tend to exhibit an even higher conversion rate), we will need to use two tables: `Call` and `Customer`. This is because the column `ProductSold` lies only in the `Call` table, yet information about customer professions and age only lie in the `Customer` table.

`SELECT` commands are not restricted to a single table. In fact, theoretically, there is no limit to the number of tables that you can extract data from in a single SQL query. Let's introduce some new concepts that are relevant once we go beyond a single table.

**Primary and foreign keys** are very important concepts that need to be understood by any database professional. Primary keys:

1. Uniquely identify a record in the table. Their name usually includes the word "ID"
    * For example, `CustomerID` is the primary key of the `Customer` table, `AgentID` is the primary key of the `Agent` table, and `CallID` is the primary key of the `Call` table    
2. Do not accept null values. And they shouldn't, because they are being used to identify the record
3. Are limited to one per table

On the other hand, foreign keys:

1. Are a field in the table that is the primary key in another table
2. Can accept null values
3. Are not limited in any way per table
    * For example, the `Call` tables has 2 foreign keys: `AgentID` and `CustomerID` pointing to the `Agent` and `Customer` tables, respectively

### Extracting call data for customers working in engineering professions (12 min)

Let's first extract the relevant data so we can perform this analysis. Here, a **`JOIN`** clause will come in handy. 

`JOIN` clauses are used to combine data from two or more tables in the same query. For example, in the current scenario, we need to get the name of the agent involved in a call. The `Call` table contains only the `AgentID` and not the name of the agent. `JOIN` becomes useful here so we can match up the `Call` table with the `Agent` table, which does contain the name information.

Here's a diagram showing how `JOIN` (specifically, the **`INNER JOIN`**, which is the default version and the only one you will need to worry about in this case) works. Notice that only the rows with `id` of 1 and 4 are extracted because those are the only two `id`s which show up in both tables:

![Join](./images/join.png)

A `JOIN` clause consists of two parts:

1. The base `JOIN` statement, which is of the form `[Table 1] JOIN [Table 2]`. This performs a Cartesian product on the 2 tables being joined. For example, if we have Table A with 5 rows, and Table 5 with 3 rows, their Cartesian product will return 15 rows (5 x 3)
2. A `JOIN` criteria, which filters the Cartesian product's results, beginning with the `ON` keyword

Here is an example of a `JOIN` criteria in action, which is telling us to only give combinations of rows where the agent ID matches in both tables:

```SQL
SELECT CallID, A.AgentID, name
FROM Call C
JOIN Agent A ON C.AgentID = A.AgentID
ORDER BY Name DESC
```

In [25]:
query6 = """SELECT CallID, A.AgentID, name
FROM Call C
JOIN Agent A ON C.AgentID = A.AgentID
ORDER BY Name DESC"""
runQuery(query6)

,CallID,AgentID,Name
0,12,3,Todd Morrow
1,28,3,Todd Morrow
2,32,3,Todd Morrow
3,50,3,Todd Morrow
4,60,3,Todd Morrow
...,...,...,...
9934,9985,10,Agent X
9935,9986,10,Agent X
9936,9991,10,Agent X
9937,9992,10,Agent X


Note that:

1. `C` and `A` are aliases to the `Call` and `Agent` tables to avoid having to type the table name every time. Unlike with column aliasing earlier, we do not need the `AS` keyword here
2. We write `A.AgentID` instead of `AgentID` in the SELECT statement – this is because the `AgentID` column exists in both tables, so we have to tell the database which one to get the result from

### Exercise 4: (7 min)

Write a query which returns all calls made out to customers in the engineering profession, and shows whether they are over or under 30 as well as whether they ended up purchasing the product from that call.

**Answer.** One possible solution is given below:

```SQL
SELECT CallID, Cu.CustomerID, Name, ProductSold,
    CASE
        WHEN Age >= 30 THEN 'Yes'
        WHEN Age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID
WHERE Occupation LIKE '%Engineer%'
ORDER BY Name DESC
```

In [26]:
query7 = """SELECT CallID, Cu.CustomerID, Name, ProductSold,
    CASE
        WHEN Age >= 30 THEN 'Yes'
        WHEN Age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID
WHERE Occupation LIKE '%Engineer%'
ORDER BY Name DESC"""
runQuery(query7)

,CallID,CustomerID,Name,ProductSold,Over30
0,2049,421,Zachary Ruiz,0,Yes
1,2960,421,Zachary Ruiz,0,Yes
2,3365,421,Zachary Ruiz,0,Yes
3,3386,421,Zachary Ruiz,1,Yes
4,4332,421,Zachary Ruiz,0,Yes
...,...,...,...,...,...
3614,6444,622,Aaron Rose,1,No
3615,7994,622,Aaron Rose,0,No
3616,8811,622,Aaron Rose,0,No
3617,9524,622,Aaron Rose,1,No


## Analyzing the call conversion data (15 min)

Now, we want to determine whether or not customers in our desired cohort exhibit a higher sales conversion rate compared to the overall population of customers. A reasonable way to do this is to count the total number of calls to this cohort which resulted in a sale, and divide that by the total number of calls to this cohort (whether or not they resulted in a sale) to get a percentage, and then compare that with the percentage we compute from the `calls` table overall.

However, to compute these figures, we'll need to learn a bit about **aggregation functions**. An aggregation function allows you to perform a calculation on a set of values to return a single value, essentially computing some sort of summary statistic.

Aggregation queries usually look like this:

![Aggregation Queries](./images/aggregation.png)

The following are the most commonly used SQL aggregate functions:

1. `AVG()` – calculates the average of a set of values
2. `COUNT()` – counts rows in a specified table or view
3. `MIN()` – gets the minimum value in a set of values
4. `MAX()` – gets the maximum value in a set of values
5. `SUM()` – calculates the sum of values

As mentioned before, PostgreSQL as some more advanced [aggregate functions](https://www.postgresql.org/docs/9.5/functions-aggregate.html). Specifically, they have some nice ones for statistics. For example,

1. `regr_intercept(Y, X)` - Returns the intercept for the line of best fit
2. `regr_slope(Y, X)` - Returns the slope of the line of best fit
3. `corr(Y, X)` - Returns the correlation between two columns

### Exercise 5: (5 min)

Write two queries - one that computes the total sales and total calls made to customers in the engineering profession, and one that computes the same metrics for the entire customer base. What can you conclude regarding the conversion rate within the engineering customers vs. the overall customer base?

**Answer.** One possible solution is given below:

```SQL
SELECT SUM(ProductSold) AS TotalSales, COUNT(*) AS NCalls
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID
WHERE Occupation LIKE '%Engineer%'
```

In [26]:
query8 = """SELECT SUM(ProductSold) AS TotalSales, COUNT(*) AS NCalls
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID
WHERE Occupation LIKE '%Engineer%'"""
runQuery(query8)

,TotalSales,NCalls
0,760,3619


In [27]:
760/3619

0.21000276319425257

```SQL
SELECT SUM(ProductSold) AS TotalSales, COUNT(*) AS NCalls
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID
```

In [36]:
query9 = """SELECT SUM(ProductSold) AS TotalSales, COUNT(*) AS NCalls, SUM(ProductSold)*100/COUNT(*) conversion_rate
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID
where occupation like '%Engineer%'
"""
runQuery(query9)

,TotalSales,NCalls,conversion_rate
0,760,3619,21


The conversion rate for both groups is ~20.9%, indicating that engineers are not more likely to purchase our products than the overall population.

### Exercise 6: (3 min)

Write a query that computes the total sales and total calls made to customers over the age of 30. Is there a notable difference between the conversion ratio here and that of the overall customer base?

**Answer.** One possible solution is given below:

```SQL
SELECT SUM(ProductSold) AS TotalSales, COUNT(*) AS NCalls
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID
WHERE Age >= 30
```

In [33]:
query10 = """SELECT SUM(ProductSold) AS TotalSales, COUNT(*) AS NCalls
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID
WHERE Age >= 30"""
runQuery(query10)

,TotalSales,NCalls
0,659,3096


The conversion rate is ~21.1% vs. the overall ~20.9%. There may be some difference, but it is quite small so we would need to run statistical significance tests in order to validate this. (You will learn about these in future cases.)

### Exercise 7: (3 min)

How about if you look at the sales conversion rate for engineers over the age of 30?

```SQL
SELECT SUM(ProductSold) AS TotalSales, COUNT(*) AS NCalls
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID
WHERE Occupation LIKE '%Engineer%' AND Age >= 30
```

**Answer.** One possible solution is given below:

In [34]:
query11 = """SELECT SUM(ProductSold) AS TotalSales, COUNT(*) AS NCalls
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID
WHERE Occupation LIKE '%Engineer%' AND Age >= 30"""
runQuery(query11)

,TotalSales,NCalls
0,376,1816


Here, we actually observe the opposite pattern – the conversion rate is only ~20.5%.

From these numbers, we can conclude that a customer's status as an engineering professional has no positive effect on their conversion rate. On the other hand, having an age of at least 30 MAY have some effect; however, we would need to do more in-depth statistical testing to determine this.

## Evaluating our agents' performance (20 min)

Recall the second part of our business question: we need to figure out which of our agents are the most and least productive. To do this, it makes sense to determine which metrics could be related to productivity. Looking at the features present, the following seem to be reasonable:

1. The number of calls an agent made
2. The lengths of calls an agent made
3. The total number of products an agent sold

### Question: (5 min)

For any given agent, would extracting this info be a good way of quickly analyzing their productivity? Why or why not?

While the above metrics are useful, some of them are too numerous to be easiy analyzed. Specifically, the lengths of calls an agent made is a dataset that is as large as the number of calls the agent made. If the agent made many calls, it will be meaningless to just throw the entire set of call lengths at ourselves. Instead, we ought to compute some summary statistics of this metric; namely, the minimum, maximum, and mean lengths seem reasonable.

### Exercise 8: (5 min)

Write a query that returns, for each agent, the agent's name, number of calls, longest and shortest call lengths, average call length, and total number of products sold. Name the columns `AgentName`, `NCalls`, `Shortest`, `Longest`, `AvgDuration`, and `TotalSales`, and order the table by `AgentName` alphabetically. (Make sure to include the `WHERE PickedUp = 1` clause to only calculate the average across all the calls that were picked up (otherwise all the minimum durations will be 0)!)

**Answer.** One possible solution is given below:

```SQL
SELECT Name AS AgentName, COUNT(*) AS NCalls, MIN(Duration) AS Shortest, MAX(Duration) AS Longest, AVG(Duration) AS AvgDuration, SUM(ProductSold) AS TotalSales
FROM Calls C
    JOIN Agent A ON C.AgentID = A.AgentID
WHERE PickeDup = 1
GROUP BY Name
ORDER BY Name
```

In [37]:
query12 = """SELECT Name AS AgentName, COUNT(*) AS NCalls, MIN(Duration) AS Shortest, MAX(Duration) AS Longest, AVG(Duration) AS AvgDuration, SUM(ProductSold) AS TotalSales
FROM Call C
    JOIN Agent A ON C.AgentID = A.AgentID
WHERE PickeDup = 1
GROUP BY Name
ORDER BY Name"""
runQuery(query12)

,AgentName,NCalls,Shortest,Longest,AvgDuration,TotalSales
0,Agent X,640,22,334,180.975000,194
1,Angel Briggs,591,12,362,181.081218,157
2,Christopher Moreno,649,47,363,177.979969,189
3,Dana Hardy,554,49,356,177.203971,182
4,Gloria Singh,662,36,349,182.175227,209
...,...,...,...,...,...,...
6,Lisa Cordova,639,46,344,179.214397,201
7,Michele Williams,685,22,306,177.880292,198
8,Paul Nunez,648,-5,323,181.070988,194
9,Randy Moore,600,16,326,178.595000,177


## Conclusions (2 min)

In this case, you learned the basics of SQL and used it to optimize the sales operations of a financial services firm. We narrowed down our set of potentially interesting customer cohorts and were able to compute summary statistics on the sales conversion rates of those cohorts, particularly versus the mean. In particular, we learned that some of our "no-brainer" hypotheses did not pan out, which illustrates the importance of always investigating the data to validate our thoughts. We also looked at sales agent performance and were able to find the ones that were most/least productive on particular metrics.

## Takeaways (3 min)

In this case, we learned the basics of RDBMS systems and their appropriate terminology. We also built a foundation of basic SQL commands to extract data from a database. Specifically we:

1. Learned what an RDBMS is
2. Connected to a database using ```SQLAlchemy```
3. Performed ```SELECT ... FROM``` queries
4. Learned the ```WHERE```, ```ORDER BY```, ```AS```, ```DISTINCT```, ```LIKE```, ```CASE-END```, and ```JOIN```, keywords
5. Performed basic aggregation methods
  